# Analiza vpliva centralizacije na občine v Sloveniji

Cilj naloge je ugotoviti katere občine bi pridobile največ z decentralizacijo delovnih mest v Sloveniji. V nalogi bom analiziral migracije med Slovenskimi občinami in se naslajal na podatke o zaslužkih ljudi v danih občinah, njihovih migracijah ter cen nepremičnin. S temi podatki bom ugotavljal v katerih občinah bi bilo mogoče najbolj izboljšat zaslužek in standard življenja. 

## Uvoz podatkov

Na začetku naloge bom iz zbranih podatkov izračunal kazalnike, ki vplivajo na posamezno občino. S tem namenom naložimo vse podatke, ki jih bomo pri analizi uporabili.

In [49]:
import pandas as pd

# Nastavitve prikaza
pd.options.display.max_rows = 20

%matplotlib inline

# Podatki
import os.path

pot_do_podatkov = "./podatki"

nepremicnine = pd.read_csv(os.path.join(pot_do_podatkov, "nepremicnine.csv"), index_col="id")
podjetja = pd.read_csv(os.path.join(pot_do_podatkov, "podjetja.csv"))
razdalje = pd.read_csv(os.path.join(pot_do_podatkov, "razdalje.csv"), index_col="zacetek")
migracije_med_obcinami = pd.read_csv(os.path.join(pot_do_podatkov, "migracije.csv"))

obcine = pd.read_csv(os.path.join(pot_do_podatkov, "obcine.csv"), index_col="ime")

Poglejmo najprej s kakšnimi podatki bomo delali. Naložili smo podatke o nepremičninah, ki vsebujejo podatke o vrsti nepremičnine, ceni, vrsti posredovanja, številu sob in podobno. Prav tako sem zbral podatke o razdaljah med posameznimi občinami in migracijah, ki nam bodo pomagale ocenit koliko povprečno posameznik izgubi časa in denarja zaradi potrebnih poti.

Poleg teh podatkov, imamo še informacije o večjih Slovenskih podjetjih, kar nam bo pomagalo ocenit pri katera podjetja delajo največjo izgubo s trenutno ureditvijo.

Vse podatke pa bomo združevali po občinah, zato imamo tudi statistične podatke o posamezni občini.

In [50]:
nepremicnine.info()
podjetja.info()
migracije_med_obcinami.info()
razdalje.info()
obcine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24934 entries, o6358608 to o6353469
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   posredovanje   24934 non-null  object 
 1   naslov         24934 non-null  object 
 2   url            24934 non-null  object 
 3   kratek_opis    24934 non-null  object 
 4   vrsta          24933 non-null  object 
 5   regija         24933 non-null  object 
 6   upravna_enota  24933 non-null  object 
 7   obcina         24757 non-null  object 
 8   velikost       24925 non-null  float64
 9   cena           24930 non-null  float64
 10  leto           20022 non-null  float64
 11  sobe           5450 non-null   float64
 12  agencija       24934 non-null  object 
dtypes: float64(4), object(9)
memory usage: 2.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9925 entries, 0 to 9924
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------          

## Obdelava podatkov o nepremičninah

Najprej izračunajmo povprečno vrednosti nepremičnin v posamezni občini. Pri tem se osredotočimo na hiše in stanovanja, ki se prodajajo in si zabeležimo še koliko nepremičnin prištejemo k povprečju.

Cene nepremičnin nam bodo pomagale ocenit, če je razlika v ceni nepremičnine, glede na to kam ljudje iz te občine migrirajo, smiselna glede na čas in stroške poti. V idealni situaciji bi živel tam, kjer je cena nepremičnin majhna, pot kratka, zaslužek pa velik.

In [51]:
# Vsem nepremičninam izračunamo ceno kvadratnega metra.
nepremicnine["cena_na_m2"] = nepremicnine.cena / nepremicnine.velikost

# Zajem
nepremicnine_po_obcinah = nepremicnine[
    nepremicnine.vrsta.isin(["Hiša", "Stanovanje"]) & 
    (nepremicnine.posredovanje == "prodaja")
].groupby("obcina")
    
# Obdelaj in priključi podatke k občinam.
obcine["cena_na_m2"] = nepremicnine_po_obcinah["cena_na_m2"].mean()
obcine["stevilo_nepremicnin"] = nepremicnine_po_obcinah.size()

## Obdelava podatkov o migracijah

Izračunajmo sedaj kako se premikajo migracije po Slovenskih regijah. Zanima nas, kakšen delež migrantov iz posamezne regije gre povprečno v dano drugo regijo. Povedano drugače, zanima nas kakšna je verjetnost, da oseba, ki živi v Ljubljani in migrira, migrira v Maribor, Koper...

To bomo izračunali na sledeč način:

1. Najprej bomo izračunali absolutno število ljudi, ki vsak dan migrira v Ljubljano. Ta podatek dobimo kot razliko delavno aktivnih prebivalcev, ki delajo v Ljubljani (po delovnem mestu) in delavno aktivnih prebivalcev, ki živijo v Ljubljani (po prebivališču), pomnožen z deležem lokalnih delavcev.

1. Glede na izbrano občino bomo izračunali kakšna je verjetnost, da gre ta oseba v druge občine. To bomo določili kot relativno število migracije v posamezno občino glede na vse ostale občine razen te, iz katere oseba prihaja. Ker ljudje raje delajo bližje doma, bom vrednosti obtežil glede na oddaljenost od dane občine.


In [71]:
migracije_po_obcinah = migracije_med_obcinami.groupby('obcina').mean()

# Dodaj podatke v občine.
obcine["delez_migrantov"] = migracije_po_obcinah.delez_migrantov
obcine["delez_lokalnih_delavcev"] = migracije_po_obcinah.delez_lokalnih_delavcev

# Koliko ljudi pride vsak dan v to občino iz druih Slovenskih občin.
obcine["st_migrantov_v_obcino"] = obcine.st_delovno_aktivnih_po_delovnem_mestu - (obcine.st_delovno_aktivnih_po_prebivaliscu * (obcine.delez_lokalnih_delavcev / 100))

# Za vsako občino dobimo podatek o tem koliko časa (sekund) in kakšno razdaljo (metrov)
# potrebuje oseba za pot do vseh ostalih občin.
def razdalja_in_cas_iz_obcine(obcina):
    return razdalje[
        (razdalje.index == obcina) &
        # nekateri podatki imajo vrednost 1000km, zato jih filtriramo.
        (razdalje.razdalja < 500_000)
    ].set_index("konec")

# Kakšna je verjetnost, da migrira nekdo iz dane občine v drugo občino.
def doloci_migracije(obcina):
    # Odstranimo občino v kateri smo.
    ostale_obcine = obcine[obcine.index != obcina].st_migrantov_v_obcino
    
    # Verjetnost, da se prebivalec naše občine odpelje v drugo občino.
    verjetnost = ostale_obcine / ostale_obcine.sum()
    
    # Občine otežimo glede na razdaljo od naše občine.
    oddaljenost = razdalja_in_cas_iz_obcine(obcina).razdalja
    utezi = (1 - pow((oddaljenost / oddaljenost.max()), 0.4))
    
    utezena_verjetnost = verjetnost * utezi

    return utezena_verjetnost / utezena_verjetnost.sum()
    
# Uporabimo narejeno funkcijo na vsaki občini, tako da dobimo razpredelnico o tem, 
# kakšna je verjetnost, da gre nekdo iz poljubne občine v poljubno občino.
migracije = pd.Series(
    obcine.index, 
    name="obcine", 
    index=obcine.index
).apply(
    doloci_migracije
).transpose()

# Izračunamo koliko časa in kilometrov v povprečju posameznik iz dane občine porabi.
def pot(obcina):
    migracije_obcina = migracije[obcina]

    df = pd.DataFrame({ 
        "razdalja": razdalja_in_cas_iz_obcine(obcina).razdalja, 
        "cas": razdalja_in_cas_iz_obcine(obcina).cas, 
        "migracije": migracije_obcina
    }, index=migracije_obcina.index)

    df["rel_razdalja"] = df.razdalja * df.migracije
    df["rel_cas"] = df.cas * df.migracije
    
    return pd.Series(
        [
            df["rel_razdalja"].sum(), 
            df["rel_cas"].sum()
        ], 
        index=["razdalja", "cas"]
    )


# Izračunaj na vseh občinah.
poti = pd.DataFrame(
    pd.Series(obcine.index, index=obcine.index).apply(pot),
    columns=["razdalja", "cas"]
)

# poti.sort_values("razdalja").tail(20)
# poti["Grosuplje"], poti["Maribor"]

# Dodaj k občinam
obcine["razdalja_migracije"] = poti.razdalja
obcine["cas_migracije"] = poti.cas

Na podlagi podatkov o migracijah lahko ocenimo koliko denarja posamezna občina nameni migracijam. To bom naredil tako, da bom na podlagi povprečne dolžine poti do delovnega mesta, izračunal koliko bencina porabijo ljudje za pot ter iz cene naftnih derivatov izračunal oceno potnih stroškov. Na podlagi povprečne neto plače v občini bom izračunal urno postavko delavca ter upošteval še povprečni čas vožnje na delovno mesto.

Povprečno porabo avtomobila sem pobral iz [te](https://www.iea.org/reports/fuel-consumption-of-cars-and-vans) strani in znaša malo več kot 7L/100km, kar bom zaokrožil na 7L/100km. Za ceno 1L goriva bom vzel kar 1€, saj je toliko znašala cena goriva v zadnjem letu.

Izračunal bom še znesek nadomestila, ki ga migrantje prejmejo od svojih podjetji kot obračun prevoza na delo. Povrnjen znesek iz [vira](https://mladipodjetnik.si/novice-in-dogodki/novice/obracun-prevoz-na-delo-ter-kilometrine-2) znaša 0.18€/km.

In [72]:
# Stroški prevoza
poraba_goriva = 7/100
cena_goriva = 1

delovni_dnevi_v_mescu = 4 * 5
st_potovanj = delovni_dnevi_v_mescu * 2

obcine["stroski_goriva"] = (obcine.razdalja_migracije / 1000) * poraba_goriva * cena_goriva * st_potovanj

# Čas potovanja
delovne_ure_v_mescu = delovni_dnevi_v_mescu * 8
obcine["povprecna_urna_postavka"] = obcine.placa_neto / delovne_ure_v_mescu

obcine["stroski_prevoza"] = (obcine.cas_migracije / 3600) * st_potovanj * obcine.povprecna_urna_postavka

# Povračilo potnih stroškov na prebivalca.
obcine["povracilo_stroskov_migracije"] = 0.18/1 * (obcine.razdalja_migracije / 1000) * st_potovanj

# Združimo vse stroške.
obcine["stroski_migracije_na_delovca"] = obcine.stroski_prevoza + obcine.stroski_goriva
obcine["razlika_v_stroskih_na_delovca"] = obcine.stroski_migracije_na_delovca - obcine.povracilo_stroskov_migracije

# Izračunamo koliko denarja občina izgubi zaradi migracij občanov.
obcine["st_migrantov_iz_obcine"] = obcine.st_delovno_aktivnih_po_prebivaliscu * (obcine.delez_migrantov / 100)
obcine["stroski_migracije"] = obcine.stroski_migracije_na_delovca * obcine.st_migrantov_iz_obcine

In [73]:
stroski_migracije = [
    "razdalja_migracije", 
    "stroski_goriva", 
    "cas_migracije", 
    "povprecna_urna_postavka", 
    "stroski_prevoza", 
    "stroski_migracije_na_delovca",
    "razlika_v_stroskih_na_delovca",
    "st_migrantov_iz_obcine",
    "stroski_migracije"
]

obcine.sort_values("stroski_migracije", ascending=False)[stroski_migracije]

,razdalja_migracije,stroski_goriva,cas_migracije,povprecna_urna_postavka,stroski_prevoza,stroski_migracije_na_delovca,razlika_v_stroskih_na_delovca,st_migrantov_iz_obcine,stroski_migracije
ime,,,,,,,,,
Ljubljana,62315.731120,174.484047,3026.383247,8.010375,269.360719,443.844766,-4.828498,17911.78830,7.950053e+06
Maribor,88862.051601,248.813744,3854.930669,6.780125,290.410131,539.223876,-100.582896,9883.54120,5.329441e+06
Koper/Capodistria,124689.931594,349.131808,4982.035072,7.352375,406.997668,756.129476,-141.638031,6604.99950,4.994235e+06
Kranj,56873.230999,159.245047,2780.619643,7.075000,218.587600,377.832646,-31.654617,11404.21305,4.308884e+06
Nova Gorica,122550.711037,343.141991,5050.322300,7.124688,399.799646,742.941637,-139.423482,4934.81680,3.666281e+06
...,...,...,...,...,...,...,...,...,...
Jezersko,76069.642233,212.994998,4321.888125,6.305625,302.802287,515.797285,-31.904139,242.58760,1.251260e+05
Kostel,109930.652860,307.805828,6134.946433,6.306813,429.910632,737.716460,-53.784241,163.57620,1.206729e+05
Solčava,74553.123133,208.748745,5031.454823,6.279375,351.048796,559.797541,23.015054,111.67175,6.251357e+04


## Obdelava podatkov o podjetjih

Ocenili bi radi katera podjetja bodo igrala ključno vlogo v posamezni občini pri doesganju decentralizacije. Predpostavil bom, da imajo podjejta vsa delovna mesta v občini registracije in poizkušal ocenit kakšen odstotek ljudi, ki dela v danem podjetju, prihaja iz posamezne občine.

Za ugotavljanje porazdelitve delavcev glede na občino prebivališča bom uporabil simetričen postopek zgornjemu, ko smo ugotavljali kam ljudje migrirajo.

1. Najprej bomo izračunali koliko ljudi iz posamezne občine migrira. Na primer, izračunali bomo, da 50 Mariborčanov in 100 Koperčanov migrira. Nato se bomo vprašali koliko ljudi, ki migrira v Ljubljano, prihaja iz Kopra - 2/3. To bomo razširili na vse občine in dobili porazdelitev delavcev migrantov po občinah.

1. Ker ljudje iz bližnjih občin prihajajo v občine v večjem številu, bomo verjetnost obtežili še glede na bližino posamezne občine.



## Združenje podatkov o nepremičninah in stroških migracije.


Naš cilj je ugotoviti katere občine imajo največji potencial za izboljšanje življenjskega standarda ljudi z decentralizacijo.
